<a href="https://colab.research.google.com/github/vinayaka16041999/Data-Science-ML-Projects/blob/master/Gender_Determination_by_Morphometry_of_Eyes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download the images


We can use **GoogleDriveDownloader** form **google_drive_downloader** library in Python to download the shared files from the shared Google drive link: https://drive.google.com/file/d/1f7uslI-ZHidriQFZR966_aILjlkgDN76/view?usp=sharing

The file id in the above link is: **1f7uslI-ZHidriQFZR966_aILjlkgDN76**

In [1]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1f7uslI-ZHidriQFZR966_aILjlkgDN76',
                                    dest_path='content/eye_gender_data.zip',
                                    unzip=True)

Unzipping...Done.


We have all the files from the shared Google drive link downloaded in the colab environment.

## Loading Libraries
All Python capabilities are not loaded to our working environment by default (even they are already installed in your system). So, we import each and every library that we want to use.

We chose alias names for our libraries for the sake of our convenience (numpy --> np and pandas --> pd, tensorlow --> tf).

Note: You can import all the libraries that you think will be required or can import it as you go along.

In [2]:
import pandas as pd                                     # Data analysis and manipultion tool
import numpy as np                                      # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf                                 # Deep Learning Tool
import os                                               # OS module in Python provides a way of using operating system dependent functionality
import cv2                                              # Library for image processing
from sklearn.model_selection import train_test_split    # For splitting the data into train and validation set
from sklearn.metrics import f1_score

## Loading and preparing training data


In [3]:
labels = pd.read_csv("/content/content/eye_gender_data/Training_set.csv")   # loading the labels
file_paths = [[fname, '/content/content/eye_gender_data/train/' + fname] for fname in labels['filename']]
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
train_data = pd.merge(images, labels, how = 'inner', on = 'filename')

data = []     # initialize an empty numpy array
image_size = 100      # image size taken is 100 here. one can take other size too
for i in range(len(train_data)):
  
  img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array
  data.append([new_img_array, train_data['label'][i]])

## Data Pre-processing
It is necessary to bring all the images in the same shape and size, also convert them to their pixel values because all machine learning or deep learning models accepts only the numerical data. Also we need to convert all the labels from categorical to numerical values.

In [4]:
labels = []
images = []
for i in range(len(data)):
  labels.append(data[i][1])
  images.append(data[i][0])
labels = np.array(labels)
images = np.array(images)
images = images.reshape(images.shape[0], 100, 100, 1)

In [5]:
images = images/255.0
labels = np.array([ 1 if x=='male' else 0 for x in labels])

In [6]:
from sklearn.model_selection import train_test_split

In [19]:
xtrain, xtest, ytrain, ytest = train_test_split(images, labels, test_size=0.01, random_state=0)

## Building Model & Hyperparameter tuning
Now we are finally ready, and we can train the model.


In [8]:
import tensorflow
from tensorflow.keras.layers import Conv2D,MaxPooling2D, Flatten,Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l1

In [9]:
!pip install keras-tuner
from keras.utils import np_utils
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation
from keras_tuner import RandomSearch

In [20]:
def build_model(hp):  # random search passes this hyperparameter() object 
    tun_mod = Sequential()
    tun_mod.add(Conv2D(hp.Int('input_units',min_value=50,max_value=500,step=10),(3,3),input_shape=(100,100,1)))
    tun_mod.add(MaxPooling2D(hp.Int('pool_size',min_value=1,max_value=5,step=1)))
    for i in range(hp.Int('num_conv',1,10)):
      tun_mod.add(Conv2D(hp.Int(f'conv_{i}_units',min_value=50,max_value=500,step=10), (3, 3)))
    tun_mod.add(MaxPooling2D(2))
    tun_mod.add(Flatten())
    for i in range(hp.Int('n_connections', 1, 10)):
        tun_mod.add(Dense(hp.Choice(f'n_nodes',values=[128, 256, 512, 1024]),activation='relu'))
    tun_mod.add(Dense(1,activation='sigmoid'))
    tun_mod.compile(optimizer=hp.Choice('optimizer',values=['adam','rmsprop','sgd']),loss='binary_crossentropy',metrics=['accuracy'])
    return tun_mod

In [21]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10, 
    directory = 'tuner'
)

INFO:tensorflow:Reloading Oracle from existing project tuner/untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from tuner/untitled_project/tuner0.json


In [23]:
NUM_EPOCH = 20
tuner.search(x=xtrain,
             y=ytrain,
             epochs=NUM_EPOCH,
             validation_data=(xtest,ytest))

INFO:tensorflow:Oracle triggered exit


In [ ]:
print(tuner.get_best_models()[0].summary())
print(tuner.get_best_hyperparameters()[0].values)

model = tuner.get_best_models(num_models=1)[0]
print (model.summary())
loss, accuracy = model.evaluate(xtest, ytest)
print('loss:', loss)
print('accuracy:', accuracy)
model.save('prueba1.h5')

## Predict The Output For Testing Dataset 😅
We have trained our model, evaluated it and now finally we will predict the output/target for the testing data (i.e. Test.csv).

#### Load Test Set
Load the test data on which final submission is to be made.

In [13]:
labels = pd.read_csv("/content/content/eye_gender_data/Testing_set.csv")   # loading the labels
file_paths = [[fname, '/content/content/eye_gender_data/test/' + fname] for fname in labels['filename']]
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_data = pd.merge(images, labels, how = 'inner', on = 'filename')

test = []     # initialize an empty numpy array
image_size = 100      # image size taken is 100 here. one can take other size too
for i in range(len(test_data)):
  
  img_array = cv2.imread(test_data['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array
  test.append(new_img_array)

## Data Pre-processing on test_data


In [14]:
test = np.array(test)
test = test.reshape(test.shape[0], 100, 100, 1)
test = test/255.0

### Make Prediction on Test Dataset
Time to make a submission!!!

In [15]:
best_model = tuner.get_best_models()[0]
pred = best_model.predict(xtest)

In [16]:
pred = [1 if x>=0.5 else 0 for x in pred]

In [17]:
pred = ['male' if x==1 else 'female' for x in pred]

## **How to save prediciton results locally via jupyter notebook?**
If you are working on Jupyter notebook, execute below block of codes. A file named 'submission.csv' will be created in your current working directory.

In [18]:
res = pd.DataFrame(pred)  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv",header=['label'], index = False)      # the csv file will be saved locally on the same location where this notebook is located.

# **OR,**
**If you are working on Google Colab then use the below set of code to save prediction results locally**

## **How to save prediction results locally via colab notebook?**
If you are working on Google Colab Notebook, execute below block of codes. A file named 'prediction_results' will be downloaded in your system.

In [ ]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False) 

# To download the csv file locally
from google.colab import files        
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Well Done! 👍**
You are all set to make a submission. Let's head to the **[challenge page](https://dphi.tech/challenges/4-week-deep-learning-online-bootcamp-final-assignment-sex-determination-by-morphometry-of-eyes/144/submit)** to make the submission.